In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.sparse.linalg
import matplotlib.pyplot as plt
import seaborn as sns
from cvxpy import *
%matplotlib inline
from generate_SPOC_model import *
import new_SPOC as new_SPOC
import SPOC as old_SPOC
from tqdm import tqdm_notebook

from multiprocessing.dummy import Pool as ThreadPool
from functools import partial

eigs = sp.sparse.linalg.eigs

In [2]:
def find_condition_number(n_clusters, n_iter, array):
    '''
    function to make n_iter iterations of generation P
    returns np.mean and np.std for F which was computed by 
    basic method and for transformed F (via rotation of ellipsoid)
    
    array is [n_nodes, pure_nodes_number]
    '''
    new_cond = []
    old_cond = []
    n_nodes = array[0]
    pure_nodes_number = array[1]
    
    for i in range(n_iter):
        A, Theta, B = generate_a(n_nodes=n_nodes, \
                                 n_clusters=n_clusters, pure_nodes_number=pure_nodes_number, seed=i)
        U, Lambda = old_SPOC.get_U_L(A, k=n_clusters)
        old_F, _ = old_SPOC.get_F_B(U, Lambda, r=n_clusters)
        
        Q = new_SPOC.new_get_Q(U)
        new_U, transform_matrix = new_SPOC.new_transform_U(U, Q, return_transform_matrix=True)
        new_F = np.dot(transform_matrix, old_F.T).T
        
        cond = np.linalg.cond(new_F)
        new_cond.append(cond)
        
        cond = np.linalg.cond(old_F)
        old_cond.append(cond)
    
    print("before ", np.mean(new_cond))
    print("after ", np.mean(old_cond))
    print
    
    return np.mean(new_cond), np.std(new_cond), np.mean(old_cond), np.std(old_cond)

In [3]:
'''
multithreadin version of computation
'''

# n_clusters = 5
# n_iter = 5

# array_of_nodes = np.arange(200, 2200, 200)
# array_of_pures = np.arange(20, 120, 20)

# array_of_nodes, array_of_pures = np.meshgrid(array_of_nodes, array_of_pures)
# array = np.vstack((array_of_nodes.ravel(), array_of_pures.ravel())).T

# # Make the Pool of workers
# pool = ThreadPool(4)
# function = partial(find_condition_number, n_clusters, n_iter)
# results = pool.map(function, array)


# #close the pool and wait for the work to finish 
# pool.close()
# pool.join()

# %time

'\nmultithreadin version of computation\n'

In [4]:
n_clusters = 2
n_iter = 5

array_of_nodes = np.arange(200, 2200, 200)
array_of_pures = np.arange(20, 120, 20)

array_of_nodes, array_of_pures = np.meshgrid(array_of_nodes, array_of_pures)
array = np.vstack((array_of_nodes.ravel(), array_of_pures.ravel())).T

for tmp in array:
    find_condition_number(n_clusters, n_iter, tmp)

('before ', 1.4654332663129686)
('after ', 1.3007852574039354)

('before ', 1.5965491983604616)
('after ', 1.4223955328701625)

('before ', 1.4642656540703463)
('after ', 1.4418664394770804)

('before ', 1.3013105384956565)
('after ', 1.3002292447661385)



KeyboardInterrupt: 